In [1]:
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
%matplotlib inline
import keras
from keras.models import Model
from keras.layers import Activation, Dense, Dropout, Flatten, Input, Concatenate
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K

Using TensorFlow backend.


In [2]:
TRAIN_DATA_DIR='./road_segmentation/training/'
TEST_DATA_DIR='./road_segmentation/test_set_images/'
NUM_TRAIN_FILES = 100
NUM_TEST_FILES = 50
IMG_SIZE = 400
TEST_IMG_SIZE = 608
PATCH_INPUT_SIZE = 32
PATCH_SIZE = 16
NUM_CLASSES = 2
RANDOM_SEED = 1337
BATCH_SIZE = 512

In [3]:
# Load data
X = np.zeros((NUM_TRAIN_FILES, IMG_SIZE, IMG_SIZE, 3))
for i in range(1, NUM_TRAIN_FILES+1):
    X[i-1] = mpimg.imread(TRAIN_DATA_DIR + "images/satImage_%.3d.png" % i)

Y = np.zeros((NUM_TRAIN_FILES, IMG_SIZE, IMG_SIZE))
for i in range(1, NUM_TRAIN_FILES+1):
    Y[i-1] = mpimg.imread(TRAIN_DATA_DIR + "groundtruth/satImage_%.3d.png" % i)
    
X_test = np.zeros((NUM_TEST_FILES, TEST_IMG_SIZE, TEST_IMG_SIZE, 3))
for i in range(1, NUM_TEST_FILES+1):
    X_test[i-1] = mpimg.imread(TEST_DATA_DIR + "test_%d/test_%d.png" % (i, i))

# Quantize labels
#Y = (Y > 0).astype(np.int)
        
def prep_data(X, Y, img_size, train):    
    # Cut into patches
    patches_per_axis = img_size // PATCH_SIZE
    
    # Padding
    padding = (PATCH_INPUT_SIZE - PATCH_SIZE) // 2
    X_padded = np.zeros(X.shape + np.array([0, 2*padding, 2*padding, 0]))
    X_padded[:, padding:-padding, padding:-padding, :] = X
    
    if train:
        step = PATCH_SIZE // 2
        num_samples = X.shape[0] * (patches_per_axis**2) * 4
    else:
        step = PATCH_SIZE
        num_samples = X.shape[0] * (patches_per_axis**2)
    
    X_patches = np.zeros((num_samples, PATCH_INPUT_SIZE, PATCH_INPUT_SIZE, X.shape[3]))
    Y_patches = np.zeros((num_samples, PATCH_SIZE, PATCH_SIZE, 1))
    
    idx = 0
    for i in tqdm(range(X.shape[0])):
        for x in range(padding, img_size, step):
            for y in range(padding, img_size, step):
                X_patches[idx] = X_padded[i, x-padding:x+PATCH_SIZE+padding, y-padding:y+PATCH_SIZE+padding, :]
                if Y is not None:
                    Y_patches[idx, :, :, 0] = Y[i, x-padding:x-padding+PATCH_SIZE, y-padding:y-padding+PATCH_SIZE]
                idx += 1
    
    if Y is not None:
        Y_labels = Y_patches.reshape((num_samples, -1)).mean(axis=1)
    else:
        Y_labels = None
        
    return X_patches, Y_patches, Y_labels
    
# Train/validate split
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.1, random_state=RANDOM_SEED)

X_train, Y_train, _ = prep_data(X_train, Y_train, IMG_SIZE, True)
X_valid, Y_valid, Y_valid_labels = prep_data(X_valid, Y_valid, IMG_SIZE, True)
X_test, _, _ = prep_data(X_test, None, TEST_IMG_SIZE, False)

100%|██████████| 50/50 [00:00<00:00, 81.59it/s]


In [27]:
def augment(X, Y):
    while True:
        idx = np.arange(X.shape[0])
        np.random.shuffle(idx)

        for i in range(0, X.shape[0]-BATCH_SIZE, BATCH_SIZE):
            X_batch, Y_batch = X[idx[i:i+BATCH_SIZE]], Y[idx[i:i+BATCH_SIZE]]

            for j in range(BATCH_SIZE):
                if np.random.rand() > .5:
                    X_batch[j], Y_batch[j] = X_batch[j, ::-1, :, :], Y_batch[j, ::-1, :, :]
                if np.random.rand() > .5:
                    X_batch[j], Y_batch[j] = X_batch[j, :, ::-1, :], Y_batch[j, :, ::-1, :]
            channel_shift = np.tile(np.random.rand(BATCH_SIZE, 3)[:, np.newaxis, np.newaxis, :], 
                                    (1, PATCH_INPUT_SIZE, PATCH_INPUT_SIZE, 1))
            X_batch += (channel_shift - .5) * .2
            yield X_batch, Y_batch

In [24]:
x = inp = Input(shape=X_train.shape[1:])
# 32*32*3
x = Conv2D(64, kernel_size=(3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
# 16*16*64
x = Conv2D(128, kernel_size=(3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
# 8*8*128
x = Conv2D(256, kernel_size=(3, 3), padding='same')(x)
x = BatchNormalization()(x)
c8 = x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
# 4*4*256
x = Conv2D(512, kernel_size=(3, 3), padding='same')(x)
x = BatchNormalization()(x)
c4 = x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
# 2*2*512
x = Conv2DTranspose(256, kernel_size=3, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
# 4*4*512
x = Concatenate(axis=3)([x, c4])
x = Conv2DTranspose(128, kernel_size=3, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
# 8*8*256
x = Concatenate(axis=3)([x, c8])
x = Conv2DTranspose(64, kernel_size=3, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
# 16*16*128
x = out = Conv2D(1, kernel_size=3, padding='same')(x)

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    p, r = precision(y_true, y_pred), recall(y_true, y_pred)
    return 2 * p * r / (p + r)

def f1_deconv(y_true, y_pred):
    y_true_reduced = K.round(K.mean(K.reshape(y_true, (-1, PATCH_SIZE**2)), axis=1))
    y_pred_reduced = K.round(K.mean(K.reshape(y_pred, (-1, PATCH_SIZE**2)), axis=1))
    return f1(y_true_reduced, y_pred_reduced)

model = Model(inputs=inp, outputs=out)
model.compile(loss=keras.losses.mean_squared_error,
              optimizer=keras.optimizers.Adam(),
              metrics=[f1_deconv])

# Choose filename for saved models here
checkpoint = ModelCheckpoint('model-deconv1.{epoch:02d}-{val_f1_deconv:.4f}.hdf5', 
                             monitor='val_f1_deconv', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')

In [ ]:
# Train model
# model.fit(X_train, Y_train, 
#           batch_size=BATCH_SIZE,
#           epochs=30,
#           verbose=1,
#           shuffle=True,
#           class_weight=class_weights,
#           validation_data=(X_valid, Y_valid),
#           callbacks=[checkpoint])

model.fit_generator(augment(X_train, Y_train), 
          steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
          epochs=5,
          verbose=1,
          validation_data=(X_valid, Y_valid),
          callbacks=[checkpoint])

Epoch 1/5
439/439 [==============================] - 144s - loss: 0.0748 - f1_deconv: 0.6393 - val_loss: 0.0529 - val_f1_deconv: nan
Epoch 2/5
439/439 [==============================] - 144s - loss: 0.0632 - f1_deconv: 0.7069 - val_loss: 0.0413 - val_f1_deconv: nan
Epoch 3/5
439/439 [==============================] - 145s - loss: 0.0562 - f1_deconv: 0.7450 - val_loss: 0.0882 - val_f1_deconv: nan
Epoch 4/5
439/439 [==============================] - 145s - loss: 0.0507 - f1_deconv: 0.7724 - val_loss: 0.0596 - val_f1_deconv: nan
Epoch 5/5
236/439 [===============>..............] - ETA: 63s - loss: 0.0468 - f1_deconv: 0.7939 

In [30]:
# Restore model
# model.load_weights("model1.28-0.9580.hdf5")

In [17]:
def predict(X):
    Y = model.predict(X, batch_size=BATCH_SIZE)
    return np.round(Y.reshape((X.shape[0], -1)).mean())

1

In [31]:
# Do prediction
Y_test = model.predict_classes(X_test)

patches_per_axis = TEST_IMG_SIZE // PATCH_SIZE
Y_test = Y_test.reshape((-1, patches_per_axis, patches_per_axis))

# Save predictions
f = open('prediction.csv', 'w')
print('id,prediction', file=f)
for i in range(1, NUM_TEST_FILES+1):
    for x in range(0, patches_per_axis):
        for y in range(0, patches_per_axis):
            print('%.3d_%d_%d,%d' % (i, x * PATCH_SIZE, y * PATCH_SIZE, Y_test[i-1, x, y]), file=f)
f.close()

72192/72200 [============================>.] - ETA: 0s  

In [65]:
Y_train.shape

(225000, 16, 16)

In [6]:
np.array([[1,2],[3,4]])[:, np.newaxis, np.newaxis, :].shape

(2, 1, 1, 2)

In [60]:
x=np.arange(4)

In [62]:
x[::-1]

array([3, 2, 1, 0])